#### CTE (Common Table Expression):
- A CTE is a named, temporary query that exists only for a single SQL statement. It improves readability and structure but cannot be reused across multiple queries.

#### Temporary Table:
- A temporary table is a session-scoped table that persists for the duration of the database connection. It can be queried, reused, indexed, and modified across multiple statements within that session.

#### Rule of thumb:
- Use a CTE for clarity in one query; use a temporary table when you need reuse or better performance across multiple steps.

In [ ]:
from db_connection import get_connection
from tabulate import tabulate

In [ ]:
sql = """
 SELECT * FROM orders;
"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()


headers = ["order_id", "customer_id", "order_date", "amount"]
print(tabulate(rows, headers=headers, tablefmt="psql"))



Problem: Calculate total sales per customer, then find customers who spent more than $1000.


In [ ]:
sql = """
with customer_totals as (
    select
        customer_id,
        sum(amount) as total_spent,
        count(*) as total_orders
    from orders
    group by customer_id
)

SELECT
    customer_id,
    total_orders,
    total_spent
FROM customer_totals
WHERE total_spent > 1000
ORDER BY total_spent DESC;
"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))



Problem: Calculate monthly sales trends and identify months with above-average performance.

In [ ]:
sql = """
WITH monthly_sales AS (
    SELECT
        DATE_TRUNC('month', sale_date) AS month,
        SUM(revenue) AS monthly_revenue,
        SUM(quantity) AS monthly_quantity
    FROM daily_sales
    GROUP BY DATE_TRUNC('month', sale_date)
),
average_metrics AS (
    SELECT
        AVG(monthly_revenue) AS avg_revenue,
        AVG(monthly_quantity) AS avg_quantity
    FROM monthly_sales
)
SELECT
    m.month,
    m.monthly_revenue,
    m.monthly_quantity
FROM monthly_sales m
CROSS JOIN average_metrics a
WHERE
    m.monthly_revenue > a.avg_revenue
ORDER BY m.month;
"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


SQL Example 3: CTEs for Data Quality Checks (Interview Favorite!)

In [ ]:
#  check for duplicate records
sql = """
-- Solution: Multi-step data quality check
WITH duplicate_check AS (
    -- Find potential duplicates
    SELECT
        customer_email,
        order_date,
        amount,
        COUNT(*) as duplicate_count
    FROM raw_orders
    GROUP BY customer_email, order_date, amount
    HAVING COUNT(*) > 1
),
amount_check AS (
    -- Find suspicious amounts
    SELECT
        order_id,
        customer_email,
        amount,
        CASE
            WHEN amount < 0 THEN 'negative_amount'
            WHEN amount > 10000 THEN 'unusually_high'
            ELSE 'ok'
        END as amount_flag
    FROM raw_orders
),
final_quality_report AS (
    -- Combine all checks
    SELECT
        ro.order_id,
        ro.customer_email,
        ro.order_date,
        ro.amount,
        ac.amount_flag,
        CASE WHEN dc.duplicate_count IS NOT NULL THEN 'potential_duplicate' ELSE 'unique' END as duplicate_flag
    FROM raw_orders ro
    LEFT JOIN amount_check ac ON ro.order_id = ac.order_id
    LEFT JOIN duplicate_check dc
        ON ro.customer_email = dc.customer_email
        AND ro.order_date = dc.order_date
        AND ro.amount = dc.amount
)
-- Final output: Only problematic records
SELECT *
FROM final_quality_report
WHERE amount_flag != 'ok' OR duplicate_flag = 'potential_duplicate'
ORDER BY order_id;

"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


simple exmaple 

In [ ]:
#  check for duplicate records
sql = """
with avg_score as (
    SELECT AVG(score) as avg_score
    FROM students
)

select s.*
from students s
cross join avg_score avg
WHERE s.score > avg.avg_score;

"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


### Chained CTE Logic

```sql
WITH
    cte1 AS (
        SELECT ...
    ),
    cte2 AS (
        SELECT ...
    ),
    cte3 AS (
        SELECT ...
    )
SELECT *
FROM cte3;


In [ ]:
sql =  """
with electronics as (
    select * from products where category = 'Electronics'
),
expensive_electronics as (
    select * from electronics where price > 400
),
in_stock as (
    select * from expensive_electronics where stock > 0
)

select
    product_id,
    price,
    stock
from in_stock
order by price desc;

"""

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


#### practise 
- YOUR TASK: Use a CTE to find departments with average salary > 60000
- Expected output: IT department (avg = 75000)

In [ ]:
sql = '''
WITH department_avg AS (
    SELECT
        department,
        AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
)

SELECT *
FROM department_avg
WHERE avg_salary > 60000;



'''

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


- Using the same employees table above
- YOUR TASK: 
- CTE 1: Calculate average salary per department
- CTE 2: Find the highest department average
- Final: Show which department(s) have the highest average salary

In [ ]:
sql = '''
WITH department_avg AS (
    SELECT
        department,
        AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
),
max_department_avg as (
    select
        max(avg_salary) as max_value
    from department_avg
)

SELECT
    da.department,
    da.avg_salary
FROM department_avg da
cross join max_department_avg m
WHERE da.avg_salary =  m.max_value


'''

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]

print(tabulate(rows, headers=headers, tablefmt="psql"))


- YOUR TASK: Use a CTE to calculate total revenue per product
- (quantity * price_per_unit), then find products with revenue > 100

In [ ]:
sql = '''

with total_sales_per_product as (
    select
    product,
    SUM(quantity * price_per_unit) AS total_sales
    from sales
    group by product
)

select * from total_sales_per_product where total_sales > 100

'''

In [ ]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        headers = [desc[0] for desc in cursor.description]
print(tabulate(rows, headers=headers, tablefmt="psql"))